In [165]:
import pandas as pd
import numpy as np
import os
import pickle

## Load results 

In [166]:
with open('results_summary.dict', 'rb') as f:
    stats = pickle.load(f)

In [167]:
with open("results/SimpleProblem-100-50-50-10000/PDL/ee83ab696457da1cb8ec3bfccfd931b9659d6fef/results-single-output-14-11/stats.dict", 'rb') as f:
    PDL_stats = pickle.load(f)

PDL_stats = {k: PDL_stats[k] for k in ['test_eval', 
               'test_eq_max', 'test_eq_mean', 
               'test_ineq_max', 'test_ineq_mean',
               'test_time']}

stats["simple_ineq50_eq50"]["PDL"] = PDL_stats
# print(stats["simple_ineq50_eq50"].keys())
# print(stats["simple_ineq50_eq50"]["baseline_opt_alm"].keys())
# print(stats["simple_ineq50_eq50"]["PDL"].keys())

stats["simple_ineq50_eq50"]["PDL"]["test_time"] = [PDL_stats["test_time"]]

for key in PDL_stats.keys():
    print(key)
    print(PDL_stats[key])

test_eval
[-10.862179  -10.847451  -11.050565  -11.963982  -10.98747    -9.603117
 -10.544283  -12.1074505 -11.940271   -9.911816  -11.120292  -11.502779
 -11.051134  -12.352814  -10.694733   -9.934385  -11.065599  -11.549038
 -10.714832  -11.158859  -12.140324  -10.027444  -11.528315  -10.8938465
 -11.411865  -11.652245  -10.269673  -11.759335  -10.382575  -11.463623
 -10.476161  -11.991989  -12.033416  -10.942486  -12.665846  -11.026836
 -11.397692  -10.11119   -11.518655  -11.718581  -11.661861  -11.370528
 -11.244843  -11.341752  -11.303523  -11.661243  -11.301643  -12.076436
 -10.28973   -10.84577   -11.233023  -10.74188    -9.698448  -10.713467
 -11.902397  -10.919155  -11.304286  -11.4458065 -11.707226  -10.9758415
 -10.960335  -10.125515  -10.811405  -11.923101  -11.052864  -11.568703
 -11.487837  -11.532968  -11.372226  -10.696724  -11.493241  -10.691717
 -10.683567  -10.539204  -11.934929  -11.807459   -9.972097  -10.79533
  -9.835893  -11.322069  -12.0112915  -9.428126  -11.

## Main tables

In [168]:
def get_latex(df):
    print(df.to_latex().replace('\$', '$'))

In [ ]:
import pandas as pd

def get_results_table(results_dict, metrics, keep_methods):
    d = {}
    missing_methods = []
    for method in keep_methods:
        if method in results_dict:
            if method == "PDL":
                d[method] = [
                    ' ' if 'time' in metric else 
                    '{:.2f} ({:.2f})'.format(*results_dict[method][metric]) for metric in metrics
                ]
            else:
                d[method] = [
                    '{:.3f} ({:.3f})'.format(*results_dict[method][metric]) if 'time' in metric else 
                    '{:.4f} ({:.4f})'.format(*results_dict[method][metric]) for metric in metrics
                ]
        else:
            missing_methods.append(method)
    
    # Create DataFrame from dictionary
    if d:  # Only if `d` is not empty
        df = pd.DataFrame.from_dict(d, orient='index')
        df.columns = metrics
    else:
        # Return an empty DataFrame with the correct columns if no methods are found
        df = pd.DataFrame(columns=metrics)
    
    df.index.names = ['alg']
    
    # Print missing methods if there are any
    if missing_methods:
        print(f'missing methods: {missing_methods}')
    
    # Filter to keep only the non-missing methods
    return df.loc[[x for x in keep_methods if x not in missing_methods]], missing_methods


In [170]:
def get_full_results_table(results_dict, exper_type, num=5):
    metrics = ['test_eval', 
               'test_eq_max', 'test_eq_mean', 
               'test_ineq_max', 'test_ineq_mean',
               'test_time']
    metrics_raw = [x.replace('test_', 'test_raw_') for x in metrics]
    metrics_renaming = ['Obj. value', 
                        'Max eq viol.', 'Mean eq viol.', 
                        'Max ineq viol.', 'Mean ineq viol.',
                        'Time']
    
    nn_methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'PDL', 'baseline_eq_nn']
    opt_methods = dict([
            ('simple', ['osqp', 'qpth', 'alm']), ('nonconvex', ['ipopt']), ('acopf', ['pypower'])
    ])
    opt_methods_list = ['baseline_opt_{}'.format(x) for x in opt_methods[exper_type]]
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizer (OSQP)'),
            ('baseline_opt_qpth', 'Optimizer (qpth)'),
            ('baseline_opt_alm', 'ALM'),
            ('PDL', 'PDL'),
            ('baseline_opt_ipopt', 'Optimizer (IPOPT)'),
            ('baseline_opt_pypower', 'Optimizer (PYPOWER)')
        ))
    
    df1, missing_methods = get_results_table(results_dict, metrics, nn_methods + opt_methods_list)
    
    # df2, missing_methods_2 = get_results_table(results_dict, metrics_raw, nn_methods)
    # df2.columns = metrics
    # df2.index =  ['{}-noTestCorr'.format(x) for x in df2.index]
    
    # missing_methods = missing_methods + ['{}-noTestCorr'.format(x) for x in missing_methods_2]
        
    # all_results = pd.concat([df1, df2])
    all_results = df1
    # results_ordering = opt_methods_list + ['method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
    #                    'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
    #                    'baseline_eq_nn']
    # all_results = all_results.loc[[x for x in results_ordering if x not in missing_methods]]
    all_results.index = [methods_renaming_dict[x] for x in all_results.index]
    all_results.columns = metrics_renaming
    
    return all_results

In [171]:
# print(stats['simple_ineq50_eq50'])
df_simple_5050 = get_full_results_table(stats['simple_ineq50_eq50'], exper_type='simple')
df_simple_5050

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'baseline_nn', 'baseline_eq_nn']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time
PDL,-10.86 (-10.85),0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00),
Optimizer (OSQP),-15.047 (0.000),0.000 (0.000),0.000 (0.000),0.001 (0.000),0.000 (0.000),0.001 (0.000)
Optimizer (qpth),-15.047 (0.000),0.000 (0.000),0.000 (0.000),0.000 (0.000),0.000 (0.000),0.867 (0.055)
ALM,-15.047 (0.000),0.000 (0.000),0.000 (0.000),0.000 (0.000),0.000 (0.000),0.033 (0.000)


In [172]:
df_nonconvex = get_full_results_table(stats['nonconvex'], exper_type='nonconvex')
df_nonconvex

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'baseline_nn', 'PDL', 'baseline_eq_nn', 'baseline_opt_ipopt']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time


In [173]:
df_acopf = get_full_results_table(stats['acopf'], exper_type='acopf')
df_acopf

missing methods: ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 'baseline_nn', 'PDL', 'baseline_eq_nn', 'baseline_opt_pypower']


,Obj. value,Max eq viol.,Mean eq viol.,Max ineq viol.,Mean ineq viol.,Time


## Appendix tables (simple problem)

In [174]:
def get_comparison_table(all_stats, experiments, metrics, keep_methods):
    d = {}
    for method in keep_methods:
        for metric in metrics:
            d[(method, metric)] = {}
            for experiment in experiments:
                d[(method, metric)][experiment] = '{:.2f} ({:.2f})'.format(*all_stats[experiment][method][metric])
        df = pd.DataFrame.from_dict(d, orient='index')           
    df.index.names = ['Alg', 'Metric']
    return df.iloc[df.index.isin(keep_methods, 'Alg')]

In [175]:
def get_simple_comparison_table(all_stats, vary_across):
    assert vary_across in ['ineq', 'eq'], 'vary_across should be in [ineq, eq]'
    
    if vary_across == 'ineq':
        experiments = ['simple_ineq10_eq50', 'simple_ineq30_eq50', 'simple_ineq50_eq50', 'simple_ineq70_eq50',
                       'simple_ineq90_eq50']
        experiments_renaming = ['10', '30', '50', '70', '90']
    else:
        experiments = ['simple_ineq50_eq10', 'simple_ineq50_eq30', 'simple_ineq50_eq50', 'simple_ineq50_eq70',
                       'simple_ineq50_eq90']
        experiments_renaming = ['10', '30', '50', '70', '90']
    
    methods = ['method', 'method_no_compl', 'method_no_corr', 'method_no_soft', 
               'baseline_nn', 'baseline_eq_nn', 'baseline_opt_osqp']
    methods_renaming_dict = dict((
            ('method', 'DC3'),
            ('method_no_compl', 'DC3, $\neq$'),
            ('method_no_corr', 'DC3, $\not\leq$ train'),
            ('method_no_corr-noTestCorr', 'DC3, $\not\leq$ train/test'),
            ('method_no_soft', 'DC3, no soft loss'),
            ('baseline_nn-noTestCorr', 'NN'),
            ('baseline_nn', 'NN, $\leq$ test'),
            ('baseline_eq_nn-noTestCorr', 'Eq.~NN'),
            ('baseline_eq_nn', 'Eq.~NN, $\leq$ test'),
            ('baseline_opt_osqp', 'Optimizers (OSQP, qpth)')
        ))
    
    metrics = ['test_eval', 'test_eq_max', 'test_ineq_max']
    metrics_renaming_dict = dict((('test_eval', 'Obj. val.'), ('test_eq_max', 'Max eq.'), ('test_ineq_max', 'Max ineq.')  ))
    clean_metrics = [metrics_renaming_dict[x] for x in metrics]
    metrics_raw = [x.replace('test_', 'test_raw_') if 'time' not in x else x for x in metrics]
    
    df1 = get_comparison_table(all_stats, experiments, metrics, methods)
    df1 = df1.reset_index()
    df1['Metric'] = df1['Metric'].str.replace('_obj_val', '_eval') # make naming consistent
    df1 = df1.set_index(['Alg', 'Metric'])
    
    df2 = get_comparison_table(all_stats, experiments, metrics_raw, methods[:-1])
    df2.columns = experiments
    df2 = df2.reset_index()
    df2['Alg'] = ['{}-noTestCorr'.format(x) for x in df2['Alg']]
    df2['Metric'] = df2['Metric'].str.replace('raw_', '')  # make naming consistent
    df2 = df2.set_index(['Alg', 'Metric'])
    
    all_results = pd.concat([df1, df2])
    results_ordering = ['baseline_opt_osqp', 'method', 'method_no_compl', 'method_no_corr', 'method_no_corr-noTestCorr',
                       'method_no_soft', 'baseline_nn-noTestCorr', 'baseline_nn', 'baseline_eq_nn-noTestCorr', 
                       'baseline_eq_nn']
    clean_results_ordering = [methods_renaming_dict[x] for x in results_ordering]
    all_results = all_results.iloc[all_results.index.isin(results_ordering, 'Alg')]
    all_results = all_results.reindex(pd.MultiIndex.from_product([results_ordering, metrics], names=['Alg', 'Metric']))
    all_results = all_results.reset_index()
    all_results['Alg'] = [methods_renaming_dict[x] for x in all_results['Alg']]
    all_results['Metric'] = [metrics_renaming_dict[x] for x in all_results['Metric']]
    all_results = all_results.set_index(['Alg', 'Metric'])
    all_results.columns = experiments_renaming
    all_results = all_results.reindex(pd.MultiIndex.from_product([clean_results_ordering, clean_metrics])) # remove names
    
    return all_results

In [176]:
df_simple_varyeq = get_simple_comparison_table(stats, 'eq')
df_simple_varyeq

KeyError: 'method'

In [ ]:
df_simple_varyineq = get_simple_comparison_table(stats, 'ineq')
df_simple_varyineq

KeyError: 'method'

## LaTeX 

In [ ]:
get_latex(df_simple_5050)

\begin{tabular}{lllllll}
\toprule
 & Obj. value & Max eq viol. & Mean eq viol. & Max ineq viol. & Mean ineq viol. & Time \\
\midrule
Optimizer (OSQP) & -15.05 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.001 (0.000) \\
Optimizer (qpth) & -15.05 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.867 (0.055) \\
\bottomrule
\end{tabular}



In [ ]:
get_latex(df_nonconvex)

In [ ]:
get_latex(df_acopf)

In [ ]:
get_latex(df_simple_varyeq)

In [ ]:
get_latex(df_simple_varyineq)